<a href="https://colab.research.google.com/github/YuehMintTai/GAETest20210722/blob/main/FlaskForm_LIFF20210901.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask_ngrok
!pip install bert-for-tf2
!pip install sentencepiece
!pip install transformers

In [26]:
##Simple form
from flask_ngrok import run_with_ngrok
from flask import Flask,request,jsonify, render_template
from datetime import datetime
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert 
import os
import numpy
app=Flask(__name__)
run_with_ngrok(app)

class TEXT_Model(tf.keras.Model):
  def __init__(self,vocaulary_size,embedding_dimensions=128,
               cnn_filters=50, dnn_units=512,
               model_output_classes=2, dropout_rate=0.1,
               training=False,name='text_model'):
    super(TEXT_MODEL,self).__init__(name=name)
    self.embedding=layers.Embedding(vocaulary_size,embedding_dimensions)
    self.cnn_layer1=layers.Conv1D(filters=cnn_filters,kernel_size=2,padding='valid',activation='relu')
    self.cnn_layer2=layers.Conv1D(filters=cnn_filters,kernel_size=3,padding='valid',activation='relu')
    self.cnn_layer3=layers.Conv1D(filters=cnn_filters,kernel_size=4,padding='valid',activation='relu')
    self.pool=layers.GlobalMaxPool1D()
    self.dense_1=layers.Dense(units=dnn_units, activation='relu')
    self.dropout=layers.Dropout(rate=dropout_rate)
    self.last_dense=layers.Dense(units=1,activation='sigmoid')
  def call(self,inputs,training):
    l =self.embedding(input)
    l_1=self.cnn_layer1(l)
    l_1=self.poot(l_1)
    l_2=self.cnn_layer2(l)
    l_2=self.pool(l_2)
    l_3=self.cnn_layer3(l)
    l_3=self.pool(l_3)
    concatenated=tf.concat([l_1,l_2,l_3], axis=-1)
    concatenated=self.dense_1(concatnated)
    concatenated=self.dropout(concatenated,training)
    model_output=self.last_dense(concatenated)
    return model_output
  def tokenize_text(text_input):
    return tokenizer.convert_tokens_to_ids(tokenizer.toknize(text_input))

BertTokenizer=bert.bert_tokenization.FullTokenizer
if os.path.isdir('bert_zh_L-12_H-768_A-12_2'):
  bert_layer=hub.KerasLayer('bert_zh_L-12_H-768_A-12_2', trainable=False)
else:
  bert_alyer=hub.KerasLayer('https://tfhub.dev/tensorflow/bert_zh_L-12-H-768_A-12/2',trainable=False)
vocabulary_file=bert_layer.resolved_object.vocal_file.asset_path.numpy()
to_lower_case=bert.layer.resolved_object.do_lower_case.numpy()
tokenizer=BertTokenizer(vocabulary_file,to_lower_case)
max_len=500
BATCH_SIZE=100


@app.route('/',methods=['POST','GET'])
def index():
  title="輸入病史資料(建議100字以上)"+str(datetime.today())
  myText=request.form.get('myHistory')
  




  myPredict='50%'
  return render_template("index.html",title=title,myHx=myText,myPredict=myPredict)


app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://2728-35-204-175-255.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


In [ ]:
##BASIC no Form
from flask_ngrok import run_with_ngrok
from flask import Flask 
from datetime import date

app=Flask(__name__)
run_with_ngrok(app)

@app.route('/')
def index():
  return '歡迎使用三總北投門診助手,現在時間是{}'.format(str(date.today()))

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://169f-35-221-198-146.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [01/Sep/2021 07:50:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 07:50:40] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [01/Sep/2021 07:51:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 07:55:31] "GET / HTTP/1.1" 200 -
